In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas_profiling import ProfileReport

In [3]:
df = pd.read_csv('train.csv')

In [4]:
pd.set_option('display.max_rows', 100)

## Separate features by type

In [5]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

### Select categorical features

In [6]:
cols_cat = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
        'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
        'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 
        'CentralAir', 'Electrical', 'Functional', 'GarageType', 'PavedDrive', 'MiscFeature', 'SaleType', 'SaleCondition']

In [7]:
df[cols_cat] = df[cols_cat].astype('string')

### Select numeric features

In [8]:
df.select_dtypes(exclude = ['object', 'string']).columns

Index(['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [9]:
cols_num = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF','EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice']

In [10]:
type(cols_num)

list

### Select ordinal categories (remove Id)

In [11]:
cols_ord = [col for col in df.columns if col not in cols_cat + cols_num]

In [12]:
cols_ord.remove("Id")

In [13]:
df[cols_ord] = df[cols_ord].astype('string')

In [14]:
cols_ord

['ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence']

## Impute NaNs

In [15]:
df_nans = df.isna().sum().reset_index().rename(columns={0:'num_nans', 'index':'col_names'})

In [16]:
cols_with_nans = df_nans[df_nans.num_nans != 0].col_names.to_list()

In [17]:
df[cols_with_nans].dtypes

LotFrontage     float64
Alley            string
MasVnrType       string
MasVnrArea      float64
BsmtQual         string
BsmtCond         string
BsmtExposure     string
BsmtFinType1     string
BsmtFinType2     string
Electrical       string
FireplaceQu      string
GarageType       string
GarageYrBlt     float64
GarageFinish     string
GarageQual       string
GarageCond       string
PoolQC           string
Fence            string
MiscFeature      string
dtype: object

In [18]:
cols_cat_with_nans = list(set(cols_with_nans) & set(cols_cat))

In [19]:
cols_ord_with_nans = list(set(cols_with_nans) & set(cols_ord))

In [20]:
cols_num_with_nans = list(set(cols_with_nans) & set(cols_num))

 ### Fill missing values in categorical features

In [21]:
df[cols_cat_with_nans] = df[cols_cat_with_nans].fillna('NoInfo')

### Fill missing values in GarageYrBlt with YearBuilt. The missing values are from variables with no garage taken into account in other feature.

In [22]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['YearBuilt'])

### Fill missing values in LotFrontage by using LotArea

In [23]:
df['LotFrontage'] = df['LotFrontage'].fillna(np.sqrt(df["LotArea"])/np.mean(np.sqrt(df["LotArea"])/(df["LotFrontage"])))

### Fill missing values in ordinal features with NoInfo

In [24]:
df[cols_ord_with_nans] = df[cols_ord_with_nans].fillna('NoInfo')

### Eliminate the rows with remaining NaNs

In [25]:
df = df.dropna(axis=0, how='any')

## Encode Categorical features with dummy variables

In [26]:
from sklearn.preprocessing import OneHotEncoder

In [27]:
enc_df = pd.get_dummies(df, columns=cols_cat, prefix=cols_cat, drop_first=True)

## Encode ordinal categories
### Create train and test subsets for Ordinal Encoding

In [28]:
data = df[cols_ord+['SalePrice']].values
x_enc = df[cols_ord]
y_enc = df['SalePrice']
print('Input', x_enc.shape)
print('Output', y_enc.shape)

Input (1452, 13)
Output (1452,)


### Fit and encode

In [29]:
map_ord = [['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'No', 'Mn', 'Av', 'Gd'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Unf', 'RFn', 'Fin'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
           ['NoInfo', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']]           

In [30]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
ordinal_enc = OrdinalEncoder(categories = map_ord)
ordinal_enc.fit(x_enc)

OrdinalEncoder(categories=[['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'No', 'Mn', 'Av', 'Gd'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Unf', 'RFn', 'Fin'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                           ['NoInfo', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']])

In [31]:
ordinal_enc.categories_

[array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'No', 'Mn', 'Av', 'Gd'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Unf', 'RFn', 'Fin'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], dtype=object),
 array(['NoInfo', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv'], dtype=object)]

In [32]:
x_enc = ordinal_enc.transform(x_enc)

In [33]:
enc_df[cols_ord] = x_enc

In [34]:
y_enc = np.log(1+enc_df['SalePrice'])

In [35]:
print(y_enc, enc_df['SalePrice'])

0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1452, dtype: float64 0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1452, dtype: int64


## Separate train and test subsets(and transform target variable)

In [36]:
from sklearn.model_selection import train_test_split
data = enc_df.to_numpy()
Id = enc_df.pop('Id')
y = np.log(1+enc_df.pop('SalePrice'))
X = enc_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 

In [37]:
pd.set_option('display.max_rows', 500)
X.shape

(1452, 235)

# Feature Selection with RandomForestRegressor and Recursive Feature Elimination

In [38]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
clf = RandomForestRegressor(n_estimators = 100, random_state=42)
sel = RFE(estimator=clf, n_features_to_select=0.1, step=1)
sel.fit(X_train, y_train)

CPU times: user 12min 6s, sys: 578 ms, total: 12min 6s
Wall time: 12min 6s


RFE(estimator=RandomForestRegressor(random_state=42), n_features_to_select=0.1)

In [55]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

23

In [56]:
selected_feat

Index(['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'BsmtQual', 'BsmtFinSF1',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'KitchenQual', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'OpenPorchSF', 'MoSold', 'GarageType_Detchd'],
      dtype='object')

In [57]:
sel_train = X_train[selected_feat]
sel_test = X_test[selected_feat]

# Train Regression Models with Selected Features

### LinearRegression

In [58]:
from sklearn.linear_model import LinearRegression
linreg=LinearRegression()
linreg.fit(sel_train,y_train)
y_linreg=linreg.predict(sel_test)

### Ridge Regression

In [59]:
from sklearn.linear_model import Ridge
ridgereg=Ridge()
ridgereg.fit(sel_train,y_train)
y_ridge=ridgereg.predict(sel_test)

### Lasso Regression

In [60]:
from sklearn.linear_model import Lasso
lassoreg=Lasso()
lassoreg.fit(sel_train,y_train)
y_lasso=lassoreg.predict(sel_test)

### Random Forest Regression

In [61]:
from sklearn.ensemble import RandomForestRegressor
rfreg=RandomForestRegressor()
rfreg.fit(sel_train,y_train)
y_rf=rfreg.predict(sel_test)

# Model Evaluation with MSE and $R^2$

In [105]:
from sklearn.metrics import mean_squared_error, r2_score
linmse=mean_squared_error(y_test,y_linreg)
ridgemse=mean_squared_error(y_test,y_ridge)
lassomse=mean_squared_error(y_test, y_lasso)
rfmse=mean_squared_error(y_test,y_rf)
linr2=r2_score(y_test,y_linreg)
ridger2=r2_score(y_test,y_ridge)
lassor2=r2_score(y_test,y_lasso)
rfr2=r2_score(y_test,y_rf)
print('Linear Regression: MSE= ', linmse,'  | R2= ',linr2)
print('Ridge Regression:  MSE= ', ridgemse, ' | R2= ', ridger2)
print('Lasso Regression:  MSE= ', lassomse, ' | R2= ', lassor2)
print('Random Forest Reg: MSE= ', rfmse, ' | R2= ', rfr2)

Linear Regression: MSE=  0.02721808514074553   | R2=  0.8329168422123578
Ridge Regression:  MSE=  0.027194052952456582  | R2=  0.8330643681638383
Lasso Regression:  MSE=  0.035627424670156733  | R2=  0.7812945845767869
Random Forest Reg: MSE=  0.024337197706479758  | R2=  0.8506016928276308


# Preprocess target dataset

In [127]:
test_df = pd.read_csv('test.csv')

Encode and impute NaNs using the methods used with training dataset.

In [128]:
enc_test = pd.get_dummies(test_df, columns=cols_cat, prefix=cols_cat, drop_first=True)
red_test = pd.DataFrame(enc_test[selected_feat])
red_test = red_test.join(enc_test['Id'])
red_test

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,BsmtFinSF1,...,GrLivArea,KitchenQual,GarageYrBlt,GarageFinish,GarageCars,GarageArea,OpenPorchSF,MoSold,GarageType_Detchd,Id
0,80.0,11622,5,6,1961,1961,0.0,TA,TA,468.0,...,896,TA,1961.0,Unf,1.0,730.0,0,6,0,1461
1,81.0,14267,6,6,1958,1958,108.0,TA,TA,923.0,...,1329,Gd,1958.0,Unf,1.0,312.0,36,6,0,1462
2,74.0,13830,5,5,1997,1998,0.0,TA,Gd,791.0,...,1629,TA,1997.0,Fin,2.0,482.0,34,3,0,1463
3,78.0,9978,6,6,1998,1998,20.0,TA,TA,602.0,...,1604,Gd,1998.0,Fin,2.0,470.0,36,6,0,1464
4,43.0,5005,8,5,1992,1992,0.0,Gd,Gd,263.0,...,1280,Gd,1992.0,RFn,2.0,506.0,82,1,0,1465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,21.0,1936,4,7,1970,1970,0.0,TA,TA,0.0,...,1092,TA,NaN,NaN,0.0,0.0,0,6,0,2915
1455,21.0,1894,4,5,1970,1970,0.0,TA,TA,252.0,...,1092,TA,1970.0,Unf,1.0,286.0,24,4,0,2916
1456,160.0,20000,5,7,1960,1996,0.0,TA,TA,1224.0,...,1224,TA,1960.0,Unf,2.0,576.0,0,9,1,2917
1457,62.0,10441,5,5,1992,1992,0.0,TA,Gd,337.0,...,970,TA,NaN,NaN,0.0,0.0,32,7,0,2918


In [129]:
red_test['LotFrontage'] = red_test['LotFrontage'].fillna(np.sqrt(red_test["LotArea"])/np.mean(np.sqrt(red_test["LotArea"])/(red_test["LotFrontage"])))

In [130]:
red_test['GarageYrBlt'] = red_test['GarageYrBlt'].fillna(red_test['YearBuilt'])

In [131]:
red_test['MasVnrArea'] = red_test['MasVnrArea'].fillna(0)

In [132]:
test_ord = red_test[list(set(red_test.columns) & set(cols_ord))]

In [133]:
test_ord = test_ord.fillna('NoInfo')

In [134]:
map_ord_test = [['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                ['NoInfo', 'Unf', 'RFn', 'Fin'],
                ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                ['NoInfo', 'Po', 'Fa', 'TA', 'Gd', 'Ex']]

In [135]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_enc = OrdinalEncoder(categories = map_ord_test)
ordinal_enc.fit(test_ord)
test_ord = ordinal_enc.transform(test_ord)

In [136]:
red_test[list(set(red_test.columns) & set(cols_ord))] = test_ord

In [140]:
red_test = red_test.fillna(red_test.mean())

## Predict test values

In [146]:
y_predict = rfreg.predict(red_test[selected_feat])
y_predict = np.expm1(y_predict)
df_out = pd.DataFrame({'Id':red_test['Id'], 'SalePrice':y_predict})
df_out

,Id,SalePrice
0,1461,130435.369695
1,1462,157706.897294
2,1463,176957.364361
3,1464,182917.447956
4,1465,197707.532551
...,...,...
1454,2915,78619.341564
1455,2916,83488.237392
1456,2917,159359.393162
1457,2918,118340.474228


## Write to file results

In [157]:
df_out.to_csv('KaggleSubmission2.2.2023.csv',index=None)

## Probando

In [ ]:
prof = ProfileReport(red_test)

In [ ]:
%%time
prof.to_file(output_file='profilehousetrain.html')

In [158]:
dfdfdf = pd.read_csv('KaggleSubmission2.2.2023.csv')
dfdfdf

,Id,SalePrice
0,1461,130435.369695
1,1462,157706.897294
2,1463,176957.364361
3,1464,182917.447956
4,1465,197707.532551
...,...,...
1454,2915,78619.341564
1455,2916,83488.237392
1456,2917,159359.393162
1457,2918,118340.474228
